**PART 2: AUTOMATIC DIFFERENTIATION**

In [1]:
import numpy as np
import random as rd

**Question 1**

In [3]:
def relu(x):
  value = np.maximum(x, 0)

  def vjp(u):
    deriv=np.zeros(len(x))
    deriv[deriv >= 0]=1
    v = u*deriv
    return v,
  return value,vjp

In [4]:
#test
y=[4,-5]
relu(y)

(array([4, 0]), <function __main__.relu.<locals>.vjp(u)>)

***Question 2***

In [9]:
def dot(W, x):
  value = np.dot(W, x)
  if value.shape==():
    value=np.array([value])
  def vjp(u):
      
      return np.outer(u, x), W.T.dot(u)
  return value, vjp

In [10]:
def mlp2(x, W1, W2):
    p,vjp_p=dot(x,W1)
    act_p,vjp_actp=relu(p)

    q,vjp_q=dot(act_p,W2)
    act_q,vjp_actq = relu(q)

    value = act_q

    def vjp(u):
        a,=vjp_actq(u)
        vjp_wrt_W2,_=vjp_q(a)

        a,=vjp_actq(u)
        _,b=vjp_q(a)
        c,=vjp_actp(b)
        vjp_wrt_W1,vjp_wrt_x=vjp_p(c)
        return vjp_wrt_x, vjp_wrt_W1, vjp_wrt_W2

    return value, vjp

In [8]:
#test
w1=np.random.randint(0,5,size=(2,2))
w2=np.random.randint(0,5,size=(2,1))
x=np.array([1,1])
u=np.array([1,1])

value,vjp=mlp2(x,w1,w2)
print(value)
print(vjp(u))

[34]
(24.0, array([[36., 36., 24., 48.],
       [36., 36., 24., 48.]]), array([[4., 2.],
       [4., 2.]]))


***Question 3***

In [ ]:
def squared_loss(y_pred, y):

  residual = y_pred - y

  def vjp(u):
    vjp_y_pred=u*residual
    vjp_y=u*(-residual)
    return vjp_y_pred,vjp_y

  value = 0.5 * np.sum(residual ** 2)
  # The code requires every output to be an array.
  return np.array([value]), vjp

In [ ]:
#test
mlp,vjp_mlp=mlp2(x,w1,w2)
value,vjp=squared_loss(mlp,1)
print(value)
print(vjp(u))

[60.5]
(array([11, 11]), array([-11, -11]))


***Question 4***

In [ ]:
def loss(x, y, W1, W2):

    y_pred,vjp_mlp = mlp2( x, W1, W2)
    val_loss,vjp_loss = squared_loss(y_pred,y)

    def vjp(u):
      a,_=vjp_loss(u)
      return *vjp_mlp(u),a

    return val_loss, vjp

In [ ]:
#test
value,vjp=loss(x,1,w1,w2)
print(value)
print(vjp(u))

[60.5]
(10.0, array([[ 5.,  0., 15.,  5.],
       [ 5.,  0., 15.,  5.]]), array([[3., 0.],
       [3., 0.]]), array([11, 11]))


***Question 5***

In [ ]:
def mlp(x,*lay):
  vjp_products=[]
  vjp_ys=[]
  y=x

  for i in range(len(lay[0])):
    a,vjp_prod=dot(lay[0][i],y) 
    y,vjp_y=relu(a)
    vjp_products.append(vjp_prod)
    vjp_ys.append(vjp_y)

  value=y
    
  def vjp(u):
    vjps=[]
    a=u
    for i in range(len(lay[0])):
      a,=vjp_ys[len(lay[0])-1-i](a)
      b,a=vjp_products[len(lay[0])-1-i](a)
      vjps.append(b)
    vjps.append(a)
    vjps.reverse()
    return *vjps,
    
  return value,vjp